In [4]:
import numpy as np
import pandas as pd

import os
import pickle




import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
from matplotlib import pyplot as plt
from mlxtend.feature_selection import SequentialFeatureSelector as sfs
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from yellowbrick.classifier import DiscriminationThreshold, roc_auc

np.random.seed(0)


%matplotlib inline

In [5]:
df = pd.read_csv('/Users/jigisharathod/Desktop/OnlineNewsPopularity.csv', index_col=0)
df.head()

,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,...,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares
url,,,,,,,,,,,,,,,,,,,,,
http://mashable.com/2013/01/07/amazon-instant-video-browser/,731.0,12.0,219.0,0.663594,1.0,0.815385,4.0,2.0,1.0,0.0,...,0.100000,0.7,-0.350000,-0.600,-0.200000,0.500000,-0.187500,0.000000,0.187500,593
http://mashable.com/2013/01/07/ap-samsung-sponsored-tweets/,731.0,9.0,255.0,0.604743,1.0,0.791946,3.0,1.0,1.0,0.0,...,0.033333,0.7,-0.118750,-0.125,-0.100000,0.000000,0.000000,0.500000,0.000000,711
http://mashable.com/2013/01/07/apple-40-billion-app-downloads/,731.0,9.0,211.0,0.575130,1.0,0.663866,3.0,1.0,1.0,0.0,...,0.100000,1.0,-0.466667,-0.800,-0.133333,0.000000,0.000000,0.500000,0.000000,1500
http://mashable.com/2013/01/07/astronaut-notre-dame-bcs/,731.0,9.0,531.0,0.503788,1.0,0.665635,9.0,0.0,1.0,0.0,...,0.136364,0.8,-0.369697,-0.600,-0.166667,0.000000,0.000000,0.500000,0.000000,1200
http://mashable.com/2013/01/07/att-u-verse-apps/,731.0,13.0,1072.0,0.415646,1.0,0.540890,19.0,19.0,20.0,0.0,...,0.033333,1.0,-0.220192,-0.500,-0.050000,0.454545,0.136364,0.045455,0.136364,505


In [6]:

#checking missing data and viewing statistics 
display(df.info())
display(df.describe().round(3))


TypeError: Cannot interpret '<attribute 'dtype' of 'numpy.generic' objects>' as a data type

In [5]:
# testing lengths to separate the classes
value = 1400
t = df[' shares']

for i in range(6): # print and increment test value by 25, 6 times
    print(f"Number of rows below {value}: ", 
          len(df[t < value]))
    value += 25

Number of rows below 1400:  18490
Number of rows below 1425:  20082
Number of rows below 1450:  20082
Number of rows below 1475:  20082
Number of rows below 1500:  20082
Number of rows below 1525:  21405


In [6]:
# viewing column labels
df.columns

Index([' timedelta', ' n_tokens_title', ' n_tokens_content',
       ' n_unique_tokens', ' n_non_stop_words', ' n_non_stop_unique_tokens',
       ' num_hrefs', ' num_self_hrefs', ' num_imgs', ' num_videos',
       ' average_token_length', ' num_keywords', ' data_channel_is_lifestyle',
       ' data_channel_is_entertainment', ' data_channel_is_bus',
       ' data_channel_is_socmed', ' data_channel_is_tech',
       ' data_channel_is_world', ' kw_min_min', ' kw_max_min', ' kw_avg_min',
       ' kw_min_max', ' kw_max_max', ' kw_avg_max', ' kw_min_avg',
       ' kw_max_avg', ' kw_avg_avg', ' self_reference_min_shares',
       ' self_reference_max_shares', ' self_reference_avg_sharess',
       ' weekday_is_monday', ' weekday_is_tuesday', ' weekday_is_wednesday',
       ' weekday_is_thursday', ' weekday_is_friday', ' weekday_is_saturday',
       ' weekday_is_sunday', ' is_weekend', ' LDA_00', ' LDA_01', ' LDA_02',
       ' LDA_03', ' LDA_04', ' global_subjectivity',
       ' global_sentiment_p

In [7]:


# make replacements in the listed names
df.columns = df.columns.str.replace(
    ' ',
    ''
).str.replace(
    'num_', 'n'
).str.replace(
    'tokens_', ''
).str.replace(
    'is_', ''
).str.replace(
    'data_', ''
).str.replace(
    'day', ''
).str.replace(
    'week_is', ''
).str.replace(
    'ectivity', ''
).str.replace(
    'arity', ''
).str.replace(
    'entertainment', 'ent'
).str.replace(
    'reference', 'ref'
).str.replace(
    'sharess', 'shares'
).str.replace(
    'average', 'avg'
).str.replace(
    'positive', 'pos'
).str.replace(
    'negative', 'neg'
)

df.columns

Index(['timedelta', 'n_title', 'n_content', 'n_unique_tokens',
       'n_non_stop_words', 'n_non_stop_unique_tokens', 'nhrefs', 'nself_hrefs',
       'nimgs', 'nvideos', 'avg_token_length', 'nkeywords',
       'channel_lifestyle', 'channel_ent', 'channel_bus', 'channel_socmed',
       'channel_tech', 'channel_world', 'kw_min_min', 'kw_max_min',
       'kw_avg_min', 'kw_min_max', 'kw_max_max', 'kw_avg_max', 'kw_min_avg',
       'kw_max_avg', 'kw_avg_avg', 'self_ref_min_shares',
       'self_ref_max_shares', 'self_ref_avg_shares', 'week_mon', 'week_tues',
       'week_wednes', 'week_thurs', 'week_fri', 'week_satur', 'week_sun',
       'weekend', 'LDA_00', 'LDA_01', 'LDA_02', 'LDA_03', 'LDA_04',
       'global_subj', 'global_sentiment_pol', 'global_rate_pos_words',
       'global_rate_neg_words', 'rate_pos_words', 'rate_neg_words',
       'avg_pos_pol', 'min_pos_pol', 'max_pos_pol', 'avg_neg_pol',
       'min_neg_pol', 'max_neg_pol', 'title_subj', 'title_sentiment_pol',
       'abs_title_

In [ ]:

# viewing data histograms
plt.figure(figsize=(20, 10))

df.hist(bins=30, figsize= (21, 13), facecolor="blue", alpha=0.5)
plt.title(r"Histogram for Online News Shares",
    fontsize=15)

plt.show()

<Figure size 1440x720 with 0 Axes>

In [ ]:
#RemovingOutliers
#Finding Median, Q1F, Q3F, Interquartile range

sorted_shares = df.sort_values('shares') 

median = sorted_shares['shares'].median() 

first_quartile = sorted_shares['shares'].quantile(q=0.25) 

third_quartile = sorted_shares['shares'].quantile(q=0.75) 

interquartile_range = third_quartile-first_quartile

In [ ]:
#inner-fences(minor outlier) and outer-fences(major outlier)
inner_boundary1 = first_quartile-(interquartile_range*1.5) #-1835.0 

inner_boundary2 = third_quartile+(interquartile_range*1.5) #5581.0 

outer_boundary1 = first_quartile-(interquartile_range*3)   #-4616.0 

outer_boundary2 = third_quartile+(interquartile_range*3)   #8362.0

print(inner_boundary1,inner_boundary2,outer_boundary1,outer_boundary2)

In [ ]:
#Removing the outliers 

df_no_outliers = df[df["shares"]<=8362] 

mean = df_no_outliers['shares'].mean() #Without outlier treatment, mean was 3395. Removing outliers, we get 1916
mean

In [1]:
Df = df[df['shares']<=outer_boundary2]

NameError: name 'df' is not defined

In [ ]:
print(f'Data before Removing Outliers = {df.shape}')
print(f'Data after Removing Outliers = {Df.shape}')
print(f'Number of Outliers = {df.shape[0] - Df.shape[0]}')

In [ ]:
Df.hist(figsize=(20,20))
plt.show()

In [ ]:
# calculate the correlation matrix
corr = df.corr()

# plot the heatmap
plt.figure(figsize=(21, 21))
plt.title('Correlations Heatmap',
          fontdict={'fontsize': 34,
                    'color':  'orange',
                    'fontweight':  'bold'},
          alpha=.8)

sns.heatmap(corr.abs(),
            cmap="Oranges", alpha=.5,
            xticklabels=corr.columns,
            yticklabels=corr.columns
            )

plt.show()

In [ ]:
#building target variable
# creating column (value = 1 for rows where `shares` is at least 1500)
df['Shares_binary'] = np.where(df['shares']>=1500, '1', '0').astype('int64')
df.Shares_binary.dtype

In [ ]:
# viewing the first 10 rows
df[['shares', 'Shares_binary']].head(10)


In [ ]:
# saving the current - state dataframe to a csv file
df.to_csv('/Users/jigisharathod/Desktop/OnlineNewsPopularity_1.csv', header=True, index=True)

In [ ]:
# setting to view all columns
pd.set_option('display.max_columns', 999)

# opening saved data from .csv file
df_2 = pd.read_csv('/Users/jigisharathod/Desktop/OnlineNewsPopularity_1.csv',
                   index_col=0)

display(df_2.shape)
df_2.head()

In [ ]:
#Helping Index for referrings
for i in range(len(Df.columns)):
    print(f'{i} -> {Df.columns[i]}')

In [ ]:
#EDA
a,b = Df['shares'].mean(),Df['shares'].median()

In [ ]:
print(f'Mean article shares = {a}')
print(f'Median article share = {b}')

In [ ]:
Wd = Df.columns.values[30:37]
Wd

In [ ]:
Unpop=Df[Df['shares']<a]
Pop=Df[Df['shares']>=a]
Unpop_day = Unpop[Wd].sum().values
Pop_day = Pop[Wd].sum().values

fig = plt.figure(figsize = (13,5))
plt.title("Count of popular/unpopular news over different day of week (Mean)", fontsize = 16)
plt.bar(np.arange(len(Wd)), Pop_day, width = 0.3, align="center", color = 'black', \
          label = "popular")
plt.bar(np.arange(len(Wd)) - 0.3, Unpop_day, width = 0.3, align = "center", color = 'grey', \
          label = "unpopular")
plt.xticks(np.arange(len(Wd)), Wd)
plt.ylabel("Count", fontsize = 12)
plt.xlabel("Days of week", fontsize = 12)
    
plt.legend(loc = 'upper right')
plt.tight_layout()
plt.show()

In [ ]:
Unpop2=Df[Df['shares']<b]
Pop2=Df[Df['shares']>=b]
Unpop_day2 = Unpop2[Wd].sum().values
Pop_day2 = Pop2[Wd].sum().values
fig = plt.figure(figsize = (13,5))
plt.title("Count of popular/unpopular news over different day of week (Median)", fontsize = 16)
plt.bar(np.arange(len(Wd)), Pop_day2, width = 0.3, align="center", color = 'black', \
          label = "popular")
plt.bar(np.arange(len(Wd)) - 0.3, Unpop_day2, width = 0.3, align = "center", color = 'grey', \
          label = "unpopular")
plt.xticks(np.arange(len(Wd)), Wd)
plt.ylabel("Count", fontsize = 12)
plt.xlabel("Days of week", fontsize = 12)
    
plt.legend(loc = 'upper right')
plt.tight_layout()
plt.show()

In [ ]:
Dc = Df.columns.values[12:18]

In [ ]:
Unpop3=Df[Df['shares']<a]
Pop3=Df[Df['shares']>=a]
Unpop_day3 = Unpop3[Dc].sum().values
Pop_day3 = Pop3[Dc].sum().values
fig = plt.figure(figsize = (13,5))
plt.title("Count of popular/unpopular news over different data channel (Mean)", fontsize = 16)
plt.bar(np.arange(len(Dc)), Pop_day3, width = 0.3, align="center", color = 'black', \
          label = "popular")
plt.bar(np.arange(len(Dc)) - 0.3, Unpop_day3, width = 0.3, align = "center", color = 'grey', \
          label = "unpopular")
plt.xticks(np.arange(len(Dc)), Dc)
plt.ylabel("Count", fontsize = 12)
plt.xlabel("Days of week", fontsize = 12)
    
plt.legend(loc = 'upper right')
plt.tight_layout()
plt.show()

In [ ]:

# viewing most and least shared articles
pd.set_option('min_rows', 20)
df_2.shares.sort_values(ascending=False)

In [ ]:
# viewing column box plots
plt.figure(figsize=(21, 13))
plt.boxplot([df_2[col] for col in df_2.iloc[:, 0:60].columns])

plt.title("Distributions of Dataset Columns",
          fontdict={'fontsize': 34})
plt.xlabel('column indexes', fontsize=25)
plt.ylabel('values', fontsize=25)


plt.show()

In [ ]:
# spliting the dataframe outcome and predictor variables
X = df_2.drop(['shares', 'Shares_binary'], axis=1)  # features
y = df_2['Shares_binary']  # target

# splitting the dataset into 75/25 training and test sets
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.25,
                                                    random_state=0
                                                    )

In [ ]:
warnings.filterwarnings("ignore")

In [ ]:
visualizer = roc_auc(LogisticRegression(), X, y)

In [ ]:
# instantiate and fit a Decision Tree classifier
tree_clf = DecisionTreeClassifier(criterion='gini',
                                  max_depth=5,
                                  random_state=0)
tree_clf.fit(X_train, y_train)

In [7]:
# viewing test set predictions
pred = tree_clf.predict(X_test)

# viewing confusion matrix and classification report
print("\n", confusion_matrix(y_test, pred), "\n")
print(classification_report(y_test, pred))

NameError: name 'tree_clf' is not defined

In [8]:
# plotting confusion matrix heatmap
cm = confusion_matrix(y_test, pred)

sns.heatmap(cm, cmap=sns.color_palette('afmhot'), annot=True, fmt='0.4g')

plt.autoscale()

plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix", fontdict={'fontsize': 21})

plt.savefig('images/dtree-cm-heatmap.png', transparent=False)

plt.show()

NameError: name 'y_test' is not defined

In [9]:
# checking feature importances of the trained model
tree_clf.feature_importances_

NameError: name 'tree_clf' is not defined

In [10]:
# visualizing Decision Tree feature importances
def plot_feature_importances(model, heading):
    n_features = X_train.shape[1]
    plt.figure(figsize=(8, 21))
    plt.title(heading, fontdict={'fontsize': 21})
    plt.barh(range(n_features),
             model.feature_importances_,
             align='center')
    plt.yticks(np.arange(n_features), X_train.columns.values)
    plt.xlabel('Feature Importance')
    plt.ylabel('Feature')


plot_feature_importances(tree_clf, "Decision Tree Feature Importances")

NameError: name 'tree_clf' is not defined

In [11]:
# building a Random Forest classifier pipeline
rf_pipeline = Pipeline([('ss', StandardScaler()),
                        ('RF', RandomForestClassifier(random_state=0))])

# defining the grid
grid = [{'RF__max_depth': [2, 4, 6],
         'RF__min_samples_split': [2, 5, 10],
         'RF__min_samples_leaf': [1, 3, 5]}]

# defining a grid search with the estimator
# using `refit`, to update the model with optimized hyperparameters
rf_gridsearch = GridSearchCV(estimator=rf_pipeline,
                             param_grid=grid,
                             scoring='accuracy',
                             cv=5,
                             refit=True,
                             n_jobs=-1,
                             verbose=1)

# fitting the training data
rf_gridsearch.fit(X_train, y_train)

# viewing accuracy on the test set
pred = rf_gridsearch.predict(X_test)
print(f"Optimal Parameters: {rf_gridsearch.best_params_}")
print("")
print("rf_gridsearch score: ",
      rf_gridsearch.score(X_test, y_test).round(2))
print("")
print(classification_report(y_test, pred))

NameError: name 'X_train' is not defined

In [12]:
# instantiate a variable for the optimized Random Forest model
rf_model = rf_gridsearch.best_estimator_.steps[1][1]

# viewing the full model and hyperperameters
rf_model


AttributeError: 'GridSearchCV' object has no attribute 'best_estimator_'

In [ ]:

# visualizing Random Forest feature importances
plot_feature_importances(rf_model, "Random Forest Feature Importances")